#### Load custom .jsonl collection of city information extraction prompts and responses

In [1]:
from datasets import load_dataset
dataset = load_dataset("teknium/openhermes")


In [2]:
# Split the dataset into training and testing sets
train_test_split = dataset["train"].train_test_split(test_size=0.1)

# Extract the training and testing datasets
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [3]:
print(f"train dataset records: {len(train_dataset)}")
print(f"test dataset records: {len(test_dataset)}")

train dataset records: 218547
test dataset records: 24284


In [4]:
train_dataset[0]

{'instruction': 'Write a paragraph that explains the importance of recycling.',
 'output': 'Recycling is an important process that helps to mitigate the human impact on the environment. By reusing materials instead of constantly extracting new resources and discarding waste, we reduce our carbon footprint and conserve natural resources. Recycling also helps to limit the amount of waste that ends up in landfills and prevents pollution from those wastes. As our global population continues to grow and consume more resources, the importance of recycling and practicing sustainable behavior will only become more crucial. By recycling, we can all play a part in ensuring that our planet remains habitable and healthy for future generations.',
 'input': ''}

#### Load the base pre-trained model and tokenizer
- Unsloth has it's own from_pretrained method.
- "load_in_4bit" indicates that the model will be quantized with bitsandbytes NormalFloat4 data type. This is the standard data type for QLoRA fine-tuning

In [5]:
from unsloth import FastLanguageModel
import torch
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = LOAD_IN_4BIT # Use 4bit quantization to reduce memory usage. Can be False.
)

==((====))==  Unsloth: Fast Mistral patching release 2024.1
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.691 GB. Platform = Linux.
O^O/ \_/ \     Pytorch: 2.1.2. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.23.post1. FA = False.
 "-____-"     Apache 2 free license: http://github.com/unslothai/unsloth
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


#### Transform our prompt/response dataset into Alpaca prompt template format 

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Will add EOS_TOKEN, otherwise your generation will go on forever!
def format_prompts(samples):
    instructions = samples["instruction"]
    inputs       = samples["input"]
    outputs      = samples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # Input is empty as no completion history
        texts.append(text)
    return { "text" : texts, }
pass

train_dataset_in_prompt_format = train_dataset.map(format_prompts, batched = True,)
test_dataset_in_prompt_format = test_dataset.map(format_prompts, batched = True,)
print(train_dataset_in_prompt_format[0]['text'])


Map:   0%|          | 0/218547 [00:00<?, ? examples/s]

Map:   0%|          | 0/24284 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a paragraph that explains the importance of recycling.

### Input:


### Response:
Recycling is an important process that helps to mitigate the human impact on the environment. By reusing materials instead of constantly extracting new resources and discarding waste, we reduce our carbon footprint and conserve natural resources. Recycling also helps to limit the amount of waste that ends up in landfills and prevents pollution from those wastes. As our global population continues to grow and consume more resources, the importance of recycling and practicing sustainable behavior will only become more crucial. By recycling, we can all play a part in ensuring that our planet remains habitable and healthy for future generations.</s>


#### Do model patching and add fast LoRA weights
r and lora_aplha are the most important parameters in LoRA configuration.

** r is the rank of the LoRA matrices:
- A higher r-value means more trainable parameters, allowing for more expressivity. But, on the negative side, there is a compute tradeoff, and may also lead to overfitting.
- A lower r-value means less trainable parameters, it can reduce overfitting at the cost of expressiveness.


** lora_aplha is a scaling factor for LoRA weights:
- Higher alpha will put more emphasis on LoRA weights.
- Lower alpha will put reduced emphasis on LoRA weights, hence model will be more dependent on its original weights.


** Important tips:
- Golden rule: lora_aplha = 2*r, i.e., if r=128 and lora_aplha should be 256
- Both r and lora_aplha should be in 2**x value, a good range for selection will be [8, 16, 32, 64, 128, 256, 512]
- If your fine-tuning data is very different from the pre-training data of your model, I recommend selecting r and lora_aplha from the higher values from the above range and vice versa.

In [7]:

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = MAX_SEQ_LENGTH,
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#### Define training arguments
Train for MAX_STEPS with a total batch size of 24 (per_device_train_batch_size*gradient_accumulation_steps)

In [8]:
MAX_STEPS=100
from transformers import TrainingArguments
training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=6,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=10,
        logging_steps=10, 
        learning_rate=2e-4,
        eval_steps=50,
        optim='adamw_8bit',
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        weight_decay=0.1,
        max_steps=MAX_STEPS,
        warmup_ratio=0.01,
        lr_scheduler_type="linear",
)

#### Construct the model trainer
- Will train the model with TRL (Transformer Reinforcement Learning), with the SFT (Supervised Fine Tuning) trainer
- Use the text column of the dataset for training

In [9]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset_in_prompt_format,
    eval_dataset = test_dataset_in_prompt_format,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    tokenizer = tokenizer,
    args = training_arguments,
)


Map:   0%|          | 0/218547 [00:00<?, ? examples/s]

Map:   0%|          | 0/24284 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


#### Show current memory stats

In [10]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.691 GB.
4.625 GB of memory reserved.


#### Train the model

In [11]:
trainer_stats = trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 218,547
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 6
  Total optimization steps = 100
  Number of trainable parameters = 41,943,040
Unsloth: `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`


Step,Training Loss,Validation Loss
50,0.591700,0.595189
100,0.612400,0.584144


Checkpoint destination directory ./results/checkpoint-10 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Saving model checkpoint to ./results/checkpoint-10
tokenizer config file saved in ./results/checkpoint-10/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10/special_tokens_map.json
Checkpoint destination directory ./results/checkpoint-20 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Saving model checkpoint to ./results/checkpoint-20
tokenizer config file saved in ./results/checkpoint-20/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-20/special_tokens_map.json
Checkpoint destination directory ./results/checkpoint-30 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Saving model checkpoint to ./results/checkpoint-30
tokenizer config file saved in ./results/checkpoint-30/tokenizer_config.json
Special tokens file saved in ./re

#### Show final memory and time stats

In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

10170.1942 seconds used for training.
169.5 minutes used for training.
Peak reserved memory = 11.084 GB.
Peak reserved memory for training = 6.459 GB.
Peak reserved memory % of max memory = 46.786 %.
Peak reserved memory for training % of max memory = 27.264 %.


#### Inference
Infer from the model using the earlier defined Alpca prompt format, leaving response blank

In [13]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence for the next 10 numbers.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # response, leaving blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence for the next 10 numbers.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987</s>']

#### Inference with text streamer


In [14]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence for the next 10 numbers.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # response, leaving blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence for the next 10 numbers.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987</s>


#### Saving only the LoRA adapters and NOT the full model
Using Huggingface's push_to_hub for an online save or save_pretrained for a local save.

In [15]:
if False: model.save_pretrained("mistral_lora_model") # local saving
if False: model.push_to_hub("your_name/lora_model", token = "...") # Huggingface hub Online saving

In [16]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

#### Example saving as transformers architecture for VLLM
Note unsloth push_to_hub_merged is same as HF push_to_hub, with additional perf features

Be aware if I save here, then it merges into n-bit then clears the LoRAs, so will get a NoneType error if later saving the save_pretrained_gguf format

In [17]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

#### GGUF / llama.cpp Conversion
Unsloth provides native GGUF/llama.cpp save. Clones llama.cpp and default save it to q8_0. Other quants include q4_k_m. Use save_pretrained_gguf for local saving and push_to_hub_gguf for uploading to HF.

In [22]:
### GGUF / llama.cpp Conversion
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model_q4_k_m_gguf", tokenizer, quantization_method = "q4_k_m")
if True: model.push_to_hub_gguf("CorticalStack/OpenHermes-Mistral-7B-GGUF", tokenizer, quantization_method = "q4_k_m", token = HF_TOKEN)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 80.79 out of 106.1 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 57.66it/s]
tokenizer config file saved in CorticalStack/OpenHermes-Mistral-7B-GGUF/tokenizer_config.json
Special tokens file saved in CorticalStack/OpenHermes-Mistral-7B-GGUF/special_tokens_map.json
Model config MistralConfig {
  "_name_or_path": "unsloth/mistral-7b",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.37.0",
  "unsloth_version": "2024.1",
  "use_cache": true,
  "vocab_size": 32000
}

Configuration saved in CorticalStack/OpenHermes-M

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 3 checkpoint shards. You can find where each parameters has been saved in the index located at CorticalStack/OpenHermes-Mistral-7B-GGUF/model.safetensors.index.json.


Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting HF into GGUF format. This will take 3 minutes...
Loading model file CorticalStack/OpenHermes-Mistral-7B-GGUF/model-00001-of-00003.safetensors
Loading model file CorticalStack/OpenHermes-Mistral-7B-GGUF/model-00001-of-00003.safetensors
Loading model file CorticalStack/OpenHermes-Mistral-7B-GGUF/model-00002-of-00003.safetensors
Loading model file CorticalStack/OpenHermes-Mistral-7B-GGUF/model-00003-of-00003.safetensors
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=32768, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1

...

model.layers.11.self_attn.k_proj.weight          -> blk.11.attn_k.weight                     | BF16   | [1024, 4096]
model.layers.11.self_attn.o_proj.weight          -> blk.11.attn_output.weight                | BF16   | [4096, 4096]
model.layers.11.self_attn.q_proj.weight          -> blk.11.attn_q.weight                     | BF16   | [4096, 4096]
model.layers.11.self_attn.v_proj.weight          -> blk.11.attn_v.weight                     | BF16   | [1024, 4096]
model.layers.12.input_layernorm.weight           -> blk.12.attn_norm.weight                  | BF16   | [4096]
model.layers.12.mlp.down_proj.weight             -> blk.12.ffn_down.weight                   | BF16   | [4096, 14336]
model.layers.12.mlp.gate_proj.weight             -> blk.12.ffn_gate.weight                   | BF16   | [14336, 4096]
model.layers.12.mlp.up_proj.weight               -> blk.12.ffn_up.weight                     | BF16   | [14336, 4096]
model.layers.12.post_attention_layernorm.weight  -> blk.12.ffn_norm

/home/ubuntu/.conda/envs/genaidemo/lib/python3.9/subprocess.py:946: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[   1/ 291]                    token_embd.weight - [ 4096, 32000,     1,     1], type =    f16, quantizing to q4_K .. size =   250.00 MiB ->    70.31 MiB
[   2/ 291]               blk.0.attn_norm.weight - [ 4096,     1,     1,     1], type =    f32, size =    0.016 MB
[   3/ 291]                blk.0.ffn_down.weight - [14336,  4096,     1,     1], type =    f16, quantizing to q6_K .. size =   112.00 MiB ->    45.94 MiB
[   4/ 291]                blk.0.ffn_gate.weight - [ 4096, 14336,     1,     1], type =    f16, quantizing to q4_K .. size =   112.00 MiB ->    31.50 MiB
[   5/ 291]                  blk.0.ffn_up.weight - [ 4096, 14336,     1,     1], type =    f16, quantizing to q4_K .. size =   112.00 MiB ->    31.50 MiB
[   6/ 291]                blk.0.ffn_norm.weight - [ 4096,     1,     1,     1], type =    f32, size =    0.016 MB
[   7/ 291]                  blk.0.attn_k.weight - [ 4096,  1024,     1,     1], type =    f16, quantizing to q4_K .. size =     8.00 MiB ->     2.25 MiB


OpenHermes-Mistral-7B-GGUF-unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Saved to https://huggingface.co/CorticalStack/OpenHermes-Mistral-7B-GGUF
